<a href="https://colab.research.google.com/github/ozanryo/air-kerma-hvl-prediction-AI/blob/main/Dataset_Generator_kVp130.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Melakukan Mounting Drive**

---

**Melakukan Pembacaan File Dicom pada Direktori yang ada di Google Drive**

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **Menginstall Library yang Dibutuhkan**

---

In [ ]:
pip install pydicom

     |████████████████████████████████| 35.5MB 1.2MB/s 


# **Melakukan Import Library yang akan digunakan**

---

In [ ]:
#Pembacaan Dicom
import pydicom as dicom
import os
 
#Operasi Matematika dan Visualisasi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
from skimage.feature import greycomatrix, greycoprops
from skimage.io import imread
from scipy import ndimage
 
# Mengunduh Dataset
from google.colab import files
 
#Pengaturan DataFrame dan Visualisasi Dataset
import pandas as pd
from pandas.plotting import scatter_matrix
 
#Preprocessing Dataset dan Membangun Neural Network
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle

# **Membaca File Dicom pada Suatu Direktori**
---

In [ ]:
PathDicom_130 = '/content/drive/My Drive/Data/Image/Recap/Part 1/130 kVp'
lstFileDicom_130 = [] #Create an Empty List
for dirName, subdirlist, filelist in os.walk(PathDicom_130):
  for filename in filelist:
    if '.dcm' in filename.lower():
      lstFileDicom_130.append(os.path.join(dirName, filename))
 
# Get ref file
RefDs = dicom.read_file(lstFileDicom_130[0])
 
# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFileDicom_130))
 
# Load spacing values (in mm)
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))
 
# The array is sized based on 'ConstPixelDims'
ArrayDicom_130 = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)
 
# loop through all the DICOM files
for filenameDCM_130 in lstFileDicom_130:
    # read the file
    ds = dicom.read_file(filenameDCM_130)
    
    # store the raw image data
    ArrayDicom_130[:, :,lstFileDicom_130.index(filenameDCM_130)] = ds.pixel_array

# **Membuat Fungsi Tambahan**
---


In [ ]:
#Membuat fungsi untuk Pengambilan Gambar dengan Koordinat utama pada tengah Image
def crop_center(img, cropx, cropy):
  y,x = img.shape
  startx = x//2-(cropx//2)
  starty = y//2-(cropy//2)    
  return img[starty:starty+cropy,startx:startx+cropx]

In [ ]:
#Membuat fungsi untuk Pengambilan Gambar dengan Koordinat utama pada titik paling kiri dan paling atas
def crop_for_iterate(img, xcrop, ycrop, j, k):
  y, x = img.shape
  startx = 0 + j*xcrop
  starty = 0 + k*ycrop    
  cropx = startx + xcrop
  cropy = starty + ycrop
  return img[starty:cropy, startx:cropx]

In [ ]:
def multiply_array(arr, n):
    result = np.zeros(shape=(n*len(arr), 1))
 
    idx = 0
    for _ in range(n):
      for element in arr:
        result[idx, 0] = element
        idx += 1
    return result

In [ ]:
def combine_array(arr_1, arr_2, arr_3):
    result = np.zeros(shape=((len(arr_1)+ len(arr_2) + len(arr_3)),1))
 
    idx = 0
    for element in arr_1:
      result[idx, 0] = element
      idx+=1
    for element in arr_2:
      result[idx, 0] = element
      idx+=1
    for element in arr_3:
      result[idx, 0] = element
      idx+=1
    return result

# **Melakukan Pengambilan Fitur menggunakan Teknik GLCM dan Membangun Dataset**
---

In [ ]:
def Data_80(start_angle, final_angle, center_crop, crop_iterate, strides):
  homogen_80 = []
  korelasi_80 = []
  dissimilar_80 = []
  energi_80 = []
  contrast_80 = []
  asm_80 = []

  dcm_files = ArrayDicom_130.shape[2]
  total_angles = final_angle - start_angle

  ## For Phantom image that take from 80 kVp
  for i in range(dcm_files):
    img_80 = ArrayDicom_130[:, :, i]
 
    for l in range(int(start_angle), int(final_angle)): 
      rot_img_80 = ndimage.rotate(img_80, l)
      crop_utama_80 = crop_center(rot_img_80, center_crop, center_crop)
 
      for j in range(int(center_crop/strides)):
        for k in range(int(center_crop/strides)):            
          crop_80 = crop_for_iterate(crop_utama_80, crop_iterate, crop_iterate, j, k)
 
          glcm_80 = greycomatrix(crop_80, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], 2048, symmetric=True, normed=True)
    
          homogen_80.append(greycoprops(glcm_80, 'homogeneity')[0, 0])
          korelasi_80.append(greycoprops(glcm_80, 'correlation')[0, 0])
          dissimilar_80.append(greycoprops(glcm_80, 'dissimilarity')[0, 0])
          energi_80.append(greycoprops(glcm_80, 'energy')[0, 0])
          contrast_80.append(greycoprops(glcm_80, 'contrast')[0, 0])
          asm_80.append(greycoprops(glcm_80, 'ASM')[0, 0])
  
  #Penempatan Hasil Pengukuran Fitur GLCM 
  hom = np.asarray(homogen_80)
  kor = np.asarray(korelasi_80)
  diss = np.asarray(dissimilar_80)
  energi = np.asarray(energi_80)
  con = np.asarray(contrast_80)
  asm = np.asarray(asm_80)
 
  Dataset = [hom, kor, diss, energi, con, asm]
  Dataset = np.asarray(Dataset)
  Dataset = np.transpose(Dataset)
 
  #Penempatan Hasil Pengukuran Hasil Eksperimen
  kVp_80 = np.asarray([130])
  Kerma_80 = np.asarray([2])
  HVL_80 = np.asarray([2])
 
  kVp_80 = multiply_array(kVp_80, int(((center_crop/strides)*(center_crop/strides))*total_angles*dcm_files))
  Kerma_80 = multiply_array(Kerma_80, int(((center_crop/strides)*(center_crop/strides))*total_angles*dcm_files))
  HVL_80 = multiply_array(HVL_80, int(((center_crop/strides)*(center_crop/strides))*total_angles*dcm_files))

  kVp = np.asarray(kVp_80)
 
  Kerma = np.asarray(Kerma_80)
 
  HVL = np.asarray(HVL_80)
 
  Dataset = np.append(Dataset, kVp, axis=1)
  Dataset = np.append(Dataset, Kerma, axis=1)
  Dataset = np.append(Dataset, HVL, axis=1)
 
  Dataset = pd.DataFrame({'Homogeneity': Dataset[:,0], 
                          'Correlation': Dataset[:,1], 
                          'Dissimilarity': Dataset[:,2], 
                          'Energy': Dataset[:,3],
                          'Contrast': Dataset[:,4],
                          'ASM': Dataset[:,5], 
                          'Nilai kVp': Dataset[:,6], 
                          'Kerma Udara (uGy/mAs)': Dataset[:,7],
                          'Half Value Layer (mmAl)': Dataset[:,8]})
  
  return Dataset

Fungsi diatas baru dapat dilakukan pada :

1. Window Center 50 x 50 
2. Window Iterate 10 x 10
3. stride 10, 10



# **Hasil Pengukuran**
---

In [ ]:
A = Data_80(start_angle=315, 
            final_angle=360, 
            center_crop=50, 
            crop_iterate=10, 
            strides=10)

In [ ]:
A

# **Mengunduh Dataset**
---

In [ ]:
with pd.ExcelWriter('Data130_part1(315, 360).xlsx') as writer:
     A.to_excel(writer, sheet_name='Sheet', index=False)
 
files.download('Data130_part1(315, 360).xlsx')